# Explore here

In [2]:
import csv

# Nombre del archivo CSV de entrada y salida
archivo_entrada = 'libro.csv'
archivo_salida = 'libro_sin_comillas.csv'

# Abrir el archivo de entrada para lectura y el archivo de salida para escritura
with open(archivo_entrada, 'r', newline='', encoding='utf-8') as csv_entrada, \
     open(archivo_salida, 'w', newline='', encoding='utf-8') as csv_salida:

    lector_csv = csv.reader(csv_entrada)
    escritor_csv = csv.writer(csv_salida)

    for fila in lector_csv:
        nueva_fila = [campo.replace('"', '') for campo in fila]
        escritor_csv.writerow(nueva_fila)

print("Se han eliminado las comillas y se ha creado el archivo", archivo_salida)


Se han eliminado las comillas y se ha creado el archivo libro_sin_comillas.csv


------------------

In [8]:
import pandas as pd
import random
# Reemplaza 'tu_archivo.csv' con el nombre de tu archivo CSV
df = pd.read_csv(r'/workspaces/028-PROYECTO_FINAL_VIVIENDAS-main/src/csv finales/comunidades_FINAL_SINDUPLICADOS.csv')

In [9]:
# Crear un DataFrame vacío para almacenar los resultados
resultados = pd.DataFrame(columns=df.columns)

# Lista de todas las comunidades autónomas únicas en el archivo
comunidades_autonomas = df['com. autonoma'].unique()


In [10]:
# Definir el número de datos que deseas obtener por cada comunidad autónoma (1500 en este caso)
datos_por_comunidad = 1400

for comunidad in comunidades_autonomas:
    # Filtrar las filas correspondientes a la comunidad actual, excluyendo 'Ceuta' y 'Melilla'
    comunidad_df = df[(df['com. autonoma'] == comunidad) & (df['com. autonoma'] != 'Ceuta') & (df['com. autonoma'] != 'Melilla')]
    
    # Verificar si hay al menos 1500 filas en esta comunidad autónoma
    if len(comunidad_df) >= datos_por_comunidad:
        # Seleccionar aleatoriamente 1500 filas
        muestra = comunidad_df.sample(n=datos_por_comunidad, random_state=42)
        # Agregar la muestra al DataFrame de resultados
        resultados = pd.concat([resultados, muestra])
    else:
        # Si no hay suficientes datos en la comunidad autónoma, agregar todos los disponibles
        resultados = pd.concat([resultados, comunidad_df])

# Los resultados ahora contienen 1500 datos de cada comunidad autónoma, excluyendo 'Ceuta' y 'Melilla'


In [11]:
# Guardar los resultados en un nuevo archivo CSV
resultados.to_csv('comunidades_FINAL_1500.csv', index=False)


---

---

In [11]:
import pandas as pd

# Ruta del archivo CSV original
ruta_archivo_original = "/workspaces/028-PROYECTO_FINAL_VIVIENDAS-main/src/PROVINCIAS/csv_brutos/a_coruña.csv"

# Leer el archivo CSV original en un DataFrame
dataframe_original = pd.read_csv(ruta_archivo_original)

# Seleccionar las columnas necesarias
columnas_seleccionadas = ["Tipo de inmueble", "Descripción", "Dirección", "Precio", "Habitaciones"]
dataframe_nuevo = dataframe_original[columnas_seleccionadas]

# Ruta del nuevo archivo CSV
ruta_archivo_nuevo = "a_coruña_clean.csv"  # Reemplaza con la ruta donde deseas guardar el nuevo archivo

# Guardar el nuevo DataFrame en un nuevo archivo CSV
dataframe_nuevo.to_csv(ruta_archivo_nuevo, index=False)

print("Nuevo archivo CSV creado con éxito.")


Nuevo archivo CSV creado con éxito.


---